In [1]:
import re 
import bisect

import pymorphy2

from collections import defaultdict, namedtuple, Counter
from itertools import chain

from tqdm import tqdm, tqdm_notebook

DocEntry = namedtuple('DocEntry', ['doc_id', 'positions'])

class Parser:
    def __init__(self):
        self.analyzer = pymorphy2.MorphAnalyzer()    
    
    def parse_text(self, text):
        words = (word for word in re.split('\W+', text) if len(word) > 0)
        norm_form = (self.analyzer.normal_forms(word)[0] for word in words)
    
        return list(norm_form)


class InvertedIndex:    
    def __init__(self):
        self.parser = Parser()
        self.dict = defaultdict(list)
        self.texts = dict()
        
    def add_document(self, doc_id, text):
        self.texts[doc_id] = text
        
        words = self.parser.parse_text(text)
        
        word_to_entry = defaultdict(lambda: [])
        
        for pos, word in enumerate(words):
            doc_entry = word_to_entry[word]
            doc_entry.append(pos)
        
        for word, positions in word_to_entry.items():
            postings = self.dict[word]
            entry = DocEntry(doc_id, positions)
            
            i = bisect.bisect_left(postings, entry)
            postings.insert(i, entry)

    def get_postings(self, word):
        return self.dict[word]    
    
    def merge(self, other_index):
        self.texts.update(other_index.texts)
        
        # Task: calc complexity and impore merging  
        for word, postings in other_index.dict.items():
            my_postings = self.dict[word]            
            self.dict[word] = sorted(chain(my_postings, postings))    

In [2]:
from multiprocessing import Process, Queue
import zipfile
import codecs

def index_worker(in_queue, out_queue):
    index = InvertedIndex()    
    
    #ProducerConsumer pattern
    while True:
        data = in_queue.get()
        
        if data is None:
            break
            
        split = data.split('\t')
        if len(split) == 2:
            doc_id, text = split
            index.add_document(doc_id, text)
    
    out_queue.put(index)

    
def index_multiproc():
    num_workers = 3

    in_queue = Queue(maxsize=100)
    out_queue = Queue()

    index = InvertedIndex()

    workers = []
    for _ in range(num_workers):
        worker = Process(target=index_worker, args=(in_queue, out_queue))
        worker.start()

        workers.append(worker)

    with zipfile.ZipFile('data/texts.zip') as zf:
        with zf.open('texts.txt', 'r') as f:
            for i, line in tqdm_notebook(zip(range(1000), codecs.iterdecode(f, 'utf-8'))):  
                in_queue.put(line)

    for _ in range(num_workers):
        in_queue.put(None)

    for worker in workers:
        index.merge(out_queue.get())   
        
    return index

if __name__ == '__main__':
    index = index_multiproc()

A Jupyter Widget

In [3]:
import math

def tfidf(term, document_id):
    docs = index.dict[term]
    for doc in docs:
        if doc.doc_id == document_id:
            return len(doc.positions)*math.log(1000/len(index.dict[term]))
    return 0

def tfidf_in_q(term, q):
    tf = 0
    for i in q.split(' '):
        if i == term:
            tf = tf + 1
    return tf

In [4]:
import operator
import pymorphy2
#метод поиска в инвертированном индексе
def searchInInvIndex(query):
    #инициализация
    documents = set()
    sim_docs = dict() 
    words_from_query = set()
    #поиск уникальных слов в запросе и лемматизация
    analyzer = pymorphy2.MorphAnalyzer()
    words_from_query = {analyzer.normal_forms(x)[0] for x in query.split(' ')}
    print(words_from_query)
    #поиск всех документов, в которых встречаются слова из запроса
    for q in words_from_query:
        if q in index.dict:
            for doc in index.dict[q]:
                documents.add(doc.doc_id)    
    #считаем cosine simularity
    for doc in documents:
        chisl = 0
        znam1 = 0
        znam2 = 0
        for q in words_from_query:
            if q in index.dict:
                chisl = chisl + tfidf(q, doc)*tfidf_in_q(q, query)
                znam1 = znam1 + tfidf(q, doc)**2
                znam2 = znam2 + tfidf_in_q(q, query)**2
        sim_docs[doc] = chisl/((znam1*znam2)**(1/2))
    #сортируем результат
    return sorted(sim_docs.items(), key=operator.itemgetter(1))

In [9]:
searchInInvIndex('поиск пробует себя на деле :)')

{'на', 'себя', 'поиск', 'пробовать', ':)', 'дело'}


[('741', 0.0),
 ('699', 0.0),
 ('906', 0.0),
 ('53', 0.0),
 ('742', 0.0),
 ('275', 0.0),
 ('251', 0.0),
 ('267', 0.0),
 ('301', 0.0),
 ('266', 0.01317124914327372),
 ('910', 0.02028361522804416),
 ('888', 0.02028361522804416),
 ('257', 0.03943174441547433),
 ('870', 0.04049233170867761),
 ('337', 0.040492331708677616),
 ('314', 0.040492331708677616),
 ('651', 0.040492331708677616),
 ('511', 0.040492331708677616),
 ('141', 0.040492331708677616),
 ('584', 0.040492331708677616),
 ('711', 0.040492331708677616),
 ('307', 0.040492331708677616),
 ('336', 0.040492331708677616),
 ('231', 0.040492331708677616),
 ('518', 0.040492331708677616),
 ('640', 0.040492331708677616),
 ('990', 0.05054553637801257),
 ('367', 0.05054553637801257),
 ('85', 0.0738085709754276),
 ('989', 0.08039366472236326),
 ('627', 0.08039366472236326),
 ('893', 0.08039366472236326),
 ('984', 0.08039366472236326),
 ('351', 0.08039366472236326),
 ('837', 0.08039366472236326),
 ('392', 0.08039366472236326),
 ('988', 0.08039366